In [2]:
import requests
import json

import os
import re

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime

from osrsbox import items_api

https://github.com/osrsbox/osrsbox-db?tab=readme-ov-file#item-properties

- id
- name
- last_updated
- cost

In [3]:
def call_http_historical_prices(item_id = 1,interval = '24h'):
    headers = {
        'User-Agent': 'Trading Strategy Platform - @funkeryiffic on Discord',
        'From': 'email@gmail.com'
    }
    response = requests.get(f'https://prices.runescape.wiki/api/v1/osrs/timeseries?timestep={interval}&id={item_id}',headers=headers)
    stats = json.loads(response.text)
    return stats


def create_price_df(item_id,interval='24h'):
    jsonData = call_http_historical_prices(item_id,interval)
    data = pd.DataFrame(jsonData)
    if data.shape[0] == 0:
        raise HistoricalDataNotFound('This item id has no historical data available')

    if data.shape[0] < 365:
        print("DataFrame has fewer than 365 rows; some data may be missing: ", item_id)

    data['index_id'] = np.arange(len(data))
    temp = data['data']
    res = pd.json_normalize(temp)
    
    res['id'] = item_id
    return res

In [4]:
create_price_df(2,interval='6h')

,timestamp,avgHighPrice,avgLowPrice,highPriceVolume,lowPriceVolume,id
0,1733551200,174,170,3507658,1028081,2
1,1733572800,173,169,6176030,1396182,2
2,1733594400,174,170,5262952,1697552,2
3,1733616000,174,169,4187660,2257835,2
4,1733637600,173,168,3925548,1068140,2
...,...,...,...,...,...,...
360,1741348800,199,194,5395567,1440010,2
361,1741370400,199,195,7240716,1992983,2
362,1741392000,201,195,7754271,2184597,2
363,1741413600,199,195,4519548,1791514,2


In [7]:
items = items_api.load()

col_names = ['name','last_updated','cost']
items_df = pd.DataFrame(columns=col_names)

for item in items:
    if item.tradeable_on_ge and not item.noted:
        items_df.loc[item.id] = (item.name,item.last_updated,item.cost)


common_trade_idx_url = "https://oldschool.runescape.wiki/w/RuneScape:Grand_Exchange_Market_Watch/Common_Trade_Index"

html = requests.get(common_trade_idx_url).content

df_index = pd.DataFrame((pd.read_html(html)[0])['Item.1'])
df_index = df_index.rename({'Item.1':'name'},axis=1)

df_index['id'] = df_index['name'].apply(lambda x:items_df.index[items_df['name'] == x][0])

In [8]:
df_index

,name,id
0,Coal,453
1,Steel bar,2353
2,Gold ore,444
3,Nature rune,561
4,Death rune,560
5,Pure essence,7936
6,Yew logs,1515
7,Magic logs,1513
8,Cowhide,1739
9,Raw lobster,377
